In [1]:
import sys
import os

# Adicione o caminho ao sys.path
module_path = os.path.abspath(os.path.join('..', 'src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import training
import training_constants as tc
import preprocessing as pre
from sklearn.pipeline import Pipeline
import feature_selection as fs
import evaluation

In [4]:
# No notebook
data_path = '../data/new_logs_labels.csv'
X, y = pre.load_data(data_path)
y.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5489 entries, 0 to 5524
Columns: 336 entries, id_log to misc_EqSec_OpFrac_MMC_MtNumerador_total
dtypes: float64(82), int64(253), object(1)
memory usage: 14.1+ MB
<class 'pandas.core.series.Series'>
Index: 5489 entries, 0 to 5524
Series name: comportamento
Non-Null Count  Dtype 
--------------  ----- 
5489 non-null   object
dtypes: object(1)
memory usage: 85.8+ KB


0    ON TASK
1    ON TASK
2    ON TASK
3    ON TASK
4    ON TASK
Name: comportamento, dtype: object

In [5]:
test_size = 0.2  # 80% for training, 20% for testing
X_train, X_test, y_train, y_test = pre.split_train_test_data(X, y, test_size, random_state=42)


Tamanho do conjunto de treino: 4391
Tamanho do conjunto de teste: 1098


In [6]:
import pandas as pd

print("Nro de instancias de cada classe em y_train:\n")
print(pd.Series(y_train).value_counts())
print("\n\nNro de instancias de cada classe em y_test:\n")
print(pd.Series(y_test).value_counts())

Nro de instancias de cada classe em y_train:

ON TASK                 2553
ON SYSTEM                721
OFF TASK                 495
ON TASK CONVERSATION     321
ON TASK OUT              301
Name: count, dtype: int64


Nro de instancias de cada classe em y_test:

ON TASK                 606
ON SYSTEM               186
OFF TASK                134
ON TASK CONVERSATION     93
ON TASK OUT              79
Name: count, dtype: int64


In [7]:
y_train, label_encoder = pre.encode_labels(y_train)
y_test = label_encoder.transform(y_test)

In [8]:
# Pré-processar os dados uma vez
preprocessor = pre.create_preprocessor(X_train)
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [9]:
print(X_train_preprocessed[:5])

[[0.65819387 0.75862069 0.77777778 ... 0.         0.         0.        ]
 [0.99729108 0.68965517 0.55555556 ... 1.         0.         0.        ]
 [0.99854326 0.89655172 0.         ... 1.         0.         0.        ]
 [0.99462421 0.20689655 0.88888889 ... 1.         0.         0.        ]
 [0.99504288 0.27586207 1.         ... 1.         0.         0.        ]]


In [10]:
print(X_train_preprocessed.shape)
import pandas as pd

# Supondo que 'X_train_preprocessed' seja seu numpy.ndarray
df = pd.DataFrame(X_train_preprocessed)

# Agora você pode chamar .describe() no DataFrame
print(df.describe())

(4391, 346)
               0            1            2            3            4    \
count  4391.000000  4391.000000  4391.000000  4391.000000  4391.000000   
mean      0.781210     0.495080     0.498823     0.494901     0.329507   
std       0.311243     0.296815     0.322054     0.316759     0.199229   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.576877     0.241379     0.222222     0.222222     0.160142   
50%       0.994437     0.482759     0.444444     0.444444     0.323843   
75%       0.997089     0.758621     0.777778     0.777778     0.485765   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

               5            6            7            8            9    ...  \
count  4391.000000  4391.000000  4391.000000  4391.000000  4391.000000  ...   
mean      0.053974     0.003872     0.066044     0.044669     0.028543  ...   
std       0.225992     0.062108     0.248388     0.099459     0.088200  ...   
min  

In [11]:
print("y_train: ", y_train[:5])

y_train:  [1 2 2 0 2]


In [12]:
# Creating selectors dynamically
selectors = fs.create_selectors_from_map(X_train_preprocessed, y_train, fs.SELECTOR_MAP)
    
best_selector, best_name, best_params, best_score = fs.evaluate_multiple_feature_selectors_with_search(X_train_preprocessed, y_train, selectors, cv=10)
print(f"Best selector: {best_name} with params: {best_params} and score: {best_score}")

print(f"Best selector: {best_selector}")
print(f"Best params: {best_params}")
print(f"Best score: {best_score}")

# Extrair o melhor número de features
if 'feature_selection__n_features' in best_params:
    n_features_to_select = best_params['feature_selection__n_features']
elif 'feature_selection__n_components' in best_params:
    n_features_to_select = best_params['feature_selection__n_components']
else:
    n_features_to_select = None  # Outros métodos podem não ter n_features

print(f"Best number of features: {n_features_to_select}")


Best selector: rfe with params: {'feature_selection__n_features_to_select': 40} and score: 0.8375007660255381
Best selector: RFE(estimator=RandomForestClassifier(random_state=42), n_features_to_select=40)
Best params: {'feature_selection__n_features_to_select': 40}
Best score: 0.8375007660255381
Best number of features: None


In [13]:
# Adicionar o melhor seletor de variáveis no pipeline de treinamento
base_pipeline = Pipeline([
    ('feature_selection', best_selector)
])

In [14]:
# Bayesian optimization não está funcionando corretamente. Usar Random Search ou Grid Search
cv=10
trained_models = training.train_model(X_train_preprocessed, y_train, tc.BAYESIAN_OPTIMIZATION, base_pipeline, n_iter=100, cv=cv)



Training and evaluating Decision Tree with Bayesian Optimization:


/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [30, 1, 2] before, using random point [5, 8, 3]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [30, 1, 2] before, using random point [20, 7, 19]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [30, 1, 2] before, using random point [3, 7, 20]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [30, 1, 2] before, using random point [30, 8, 7]
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evalu

Bayesian Optimization Best Result for Decision Tree: 0.7955414902103197


In [15]:
# Caminhos
dirpath = "../output/"
model_dir = "../models/"

# Gerar e imprimir relatórios
reports = evaluation.generate_reports(trained_models, X_train_preprocessed, y_train, X_test_preprocessed, y_test)
print(evaluation.print_reports(reports, dirpath))
evaluation.save_reports_to_csv(reports, dirpath)

# Salvar todos os modelos
saved_models = evaluation.dump_all_models(trained_models, model_dir)
print("Modelos salvos:", saved_models)


Evaluating Decision Tree with Bayesian Optimization:
Hyperparameters: {'memory': None, 'steps': [('classifier', DecisionTreeClassifier())], 'verbose': False, 'classifier': DecisionTreeClassifier(), 'classifier__ccp_alpha': 0.0, 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': None, 'classifier__max_features': None, 'classifier__max_leaf_nodes': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__min_weight_fraction_leaf': 0.0, 'classifier__random_state': None, 'classifier__splitter': 'best'}

Training set report:
Class 0 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 495.0
Class 1 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 721.0
Class 2 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 2553.0
Class 3 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 321.0
Class 4 - Precision: 1.0, Recall: 1.0, F1-Score: 1.0, Support: 301.0
Accuracy - 